# PythonPDEVS examples

Import PythonPDEVS modelling elements for *AtomicDEVS* and *CoupledDEVS* models.
Also import the concept of *INFINITY* to be used.

In [1]:
from pypdevs.DEVS import AtomicDEVS, CoupledDEVS
from pypdevs.infinity import INFINITY

Create some simple *AtomicDEVS* models for a *Generator* and a *Queue*.

In [2]:
class Generator(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Generator")
        self.state = 0
        self.outport = self.addOutPort("outport")

    def timeAdvance(self):
        return 1.0

    def outputFnc(self):
        # Our message is simply the integer 5, though this could be anything
        return {self.outport: 5}

    def intTransition(self):
        return self.state + 1

In [3]:
class Queue(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Queue")
        self.state = None
        self.processing_time = 1.0
        self.inport = self.addInPort("input")
        self.outport = self.addOutPort("output")

    def timeAdvance(self):
        if self.state is None:
            return INFINITY
        else:
            return self.processing_time

    def outputFnc(self):
        return {self.outport: self.state}

    def extTransition(self, inputs):
        self.state = inputs[self.inport]
        return self.state

    def intTransition(self):
        return None

Add the Coupled DEVS model for a simple *CoupledQueue*.

In [4]:
class CQueue(CoupledDEVS):
    def __init__(self):
        CoupledDEVS.__init__(self, "CQueue")
        self.generator = self.addSubModel(Generator())
        self.queue = self.addSubModel(Queue())
        self.connectPorts(self.generator.outport, self.queue.inport)
    
    def select(self, imm):
        return self.queue

Finally run the simulation by setting up an experiment file as follows.
First, however, we have to include the simulator concept.

In [5]:
from pypdevs.simulator import Simulator

In [6]:
model = CQueue()
sim = Simulator(model)
# Required to set Classic DEVS, as we simulate in Parallel DEVS otherwise
sim.setClassicDEVS()
sim.setTerminationTime(10.0)
sim.simulate()

TypeError: unorderable types: Queue() < Generator()

Simulation is now done, and the results can be seen in the changed states of the model.

In [7]:
model.generator.state

1

Another option is to use verbose simulation, as follows.

In [8]:
model = CQueue()
sim = Simulator(model)
sim.setVerbose()
# Required to set Classic DEVS, as we simulate in Parallel DEVS otherwise
sim.setClassicDEVS()
sim.setTerminationTime(10.0)
sim.simulate()


__  Current Time:       0.00 __________________________________________ 


	INITIAL CONDITIONS in model <CQueue.Generator>
		Initial State: 0
		Next scheduled internal transition at time 1.00


	INITIAL CONDITIONS in model <CQueue.Queue>
		Initial State: None
		Next scheduled internal transition at time inf



TypeError: unorderable types: Queue() < Generator()

Apart from Classic DEVS simulation, Parallel DEVS simulation is also possible.bag
For this, we add a slightly altered version of the models, to comply with bag semantics.

In [9]:
class Generator(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Generator")
        self.state = 0
        self.outport = self.addOutPort("outport")

    def timeAdvance(self):
        return 1.0

    def outputFnc(self):
        # Our message is simply the integer 5, though this could be anything
        return {self.outport: [5]}

    def intTransition(self):
        return self.state + 1
    
class Queue(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Queue")
        self.state = None
        self.processing_time = 1.0
        self.inport = self.addInPort("input")
        self.outport = self.addOutPort("output")

    def timeAdvance(self):
        if self.state is None:
            return INFINITY
        else:
            return self.processing_time

    def outputFnc(self):
        return {self.outport: [self.state]}

    def extTransition(self, inputs):
        self.state = inputs[self.inport][0]
        return self.state

    def intTransition(self):
        self.state = None
        return self.state

And similar to before, we start the experiment, now not using the *setClassicDEVS* option.

In [10]:
model = CQueue()
sim = Simulator(model)
sim.setVerbose()
sim.setTerminationTime(10.0)
sim.simulate()


__  Current Time:       0.00 __________________________________________ 


	INITIAL CONDITIONS in model <CQueue.Generator>
		Initial State: 0
		Next scheduled internal transition at time 1.00


	INITIAL CONDITIONS in model <CQueue.Queue>
		Initial State: None
		Next scheduled internal transition at time inf


__  Current Time:       1.00 __________________________________________ 


	EXTERNAL TRANSITION in model <CQueue.Queue>
		Input Port Configuration:
			port <input>:
				5
		New State: 5
		Next scheduled internal transition at time 2.00


	INTERNAL TRANSITION in model <CQueue.Generator>
		New State: 1
		Output Port Configuration:
			port <outport>:
				5
		Next scheduled internal transition at time 2.00


__  Current Time:       2.00 __________________________________________ 


	CONFLUENT TRANSITION in model <CQueue.Queue>
		Input Port Configuration:
			port <input>: 
				5
		New State: 5
		Output Port Configuration:
			port <output>:
				5
		Next scheduled internal transition at

A termination condition can also be used.

In [11]:
def termFunc(clock, model):
    if model.generator.state > 5:
        # The generator has generated more than 5 events
        # So stop
        return True
    elif clock[0] > 10:
        # Or if the clock has progressed past simulation time 10
        return True
    else:
        # Otherwise, we simply continue
        return False

In [12]:
model = CQueue()
sim = Simulator(model)
sim.setVerbose()
sim.setTerminationCondition(termFunc)
sim.simulate()


__  Current Time:       0.00 __________________________________________ 


	INITIAL CONDITIONS in model <CQueue.Generator>
		Initial State: 0
		Next scheduled internal transition at time 1.00


	INITIAL CONDITIONS in model <CQueue.Queue>
		Initial State: None
		Next scheduled internal transition at time inf


__  Current Time:       1.00 __________________________________________ 


	EXTERNAL TRANSITION in model <CQueue.Queue>
		Input Port Configuration:
			port <input>:
				5
		New State: 5
		Next scheduled internal transition at time 2.00


	INTERNAL TRANSITION in model <CQueue.Generator>
		New State: 1
		Output Port Configuration:
			port <outport>:
				5
		Next scheduled internal transition at time 2.00


__  Current Time:       2.00 __________________________________________ 


	CONFLUENT TRANSITION in model <CQueue.Queue>
		Input Port Configuration:
			port <input>: 
				5
		New State: 5
		Output Port Configuration:
			port <output>:
				5
		Next scheduled internal transition at